Data Scraping & Package Installation

In [1]:
from unsloth import FastLanguageModel
import torch
import unsloth
from datasets import load_dataset

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


# Load LLaMA 3 model
model_name = "unsloth/Llama-3.2-1B-Instruct" 
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

/home/ubuntu/miniconda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.46.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 7.5. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


<string>:204: SyntaxWarning: invalid escape sequence '\ '
<string>:205: SyntaxWarning: invalid escape sequence '\_'
<string>:206: SyntaxWarning: invalid escape sequence '\ '
<string>:204: SyntaxWarning: invalid escape sequence '\ '
<string>:205: SyntaxWarning: invalid escape sequence '\_'
<string>:206: SyntaxWarning: invalid escape sequence '\ '


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.7 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [3]:
from datasets import load_dataset

rag_prompt_template = """Below is an user question input, paired with a retrieval context. Write a response that appropriately completes the request.

### Input:
{}

### Context:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["question"]
    contexts     = examples["context"]
    outputs      = examples["answer_body"]
    texts = []
    for input, context, output in zip(inputs, contexts, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = rag_prompt_template.format(input, context, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = load_dataset("pacovaldez/pandas-questions", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [4]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

/home/ubuntu/miniconda/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
max_steps is given, it will override any value given in num_train_epochs


In [5]:
torch.cuda.empty_cache()
def print_cuda_memory_stats(device_index=0):
    # Ensure CUDA is available
    if not torch.cuda.is_available():
        print("CUDA is not available.")
        return
    
    # Select the device
    device = torch.device(f"cuda:{device_index}")
    torch.cuda.set_device(device)
    
    # Get memory statistics
    total_memory = torch.cuda.get_device_properties(device).total_memory
    allocated_memory = torch.cuda.memory_allocated(device)
    cached_memory = torch.cuda.memory_reserved(device)
    
    # Convert bytes to gigabytes for easier interpretation
    total_memory_gb = total_memory / (1024 ** 3)
    allocated_memory_gb = allocated_memory / (1024 ** 3)
    cached_memory_gb = cached_memory / (1024 ** 3)
    
    # Print memory information
    print(f"Total memory on CUDA device {device_index}: {total_memory_gb:.2f} GB")
    print(f"Memory allocated on CUDA device {device_index}: {allocated_memory_gb:.2f} GB")
    print(f"Memory cached on CUDA device {device_index}: {cached_memory_gb:.2f} GB")

print_cuda_memory_stats(0)  # for device index 0

gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

Total memory on CUDA device 0: 14.74 GB
Memory allocated on CUDA device 0: 1.13 GB
Memory cached on CUDA device 0: 1.15 GB
GPU = Tesla T4. Max memory = 14.741 GB.
1.148 GB of memory reserved.


In [6]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,860 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 11,272,192


Step,Training Loss
1,1.798700
2,1.835000
3,1.839500
4,1.804900
5,1.911800
6,1.859300
7,1.655800
8,1.633200
9,1.739500
10,1.621200


In [15]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

423.3765 seconds used for training.
7.06 minutes used for training.
Peak reserved memory = 7.139 GB.
Peak reserved memory for training = 2.356 GB.
Peak reserved memory % of max memory = 48.43 %.
Peak reserved memory for training % of max memory = 15.983 %.


In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    rag_prompt_template.format(
        "Explain how to merge DataFrames in pandas", # input
        "Pandas merge function can be used to merge two dataframes. This function is similar to SQL join operation.", # context
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = max_seq_length)

<|begin_of_text|>Below is an user question input, paired with a retrieval context. Write a response that appropriately completes the request.

### Input:
Explain how to merge DataFrames in pandas

### Context:
Pandas merge function can be used to merge two dataframes. This function is similar to SQL join operation.

### Response:
You can use the merge function to merge two dataframes. Here is how you can do it:
df1 = pd.DataFrame({"key1": [1, 2, 3, 4, 5], "key2": [10, 20, 30, 40, 50]})

df2 = pd.DataFrame({"key1": [1, 2, 3, 4, 5], "key3": [100, 200, 300, 400, 500]})

df3 = pd.DataFrame({"key1": [1, 2, 3, 4, 5], "key2": [100, 200, 300, 400, 500]})

df4 = pd.DataFrame({"key1": [1, 2, 3, 4, 5], "key3": [100, 200, 300, 400, 500]})

df5 = pd.DataFrame({"key1": [1, 2, 3, 4, 5], "key2": [100, 200, 300, 400, 500]})

df6 = pd.DataFrame({"key1": [1, 2, 3, 4, 5], "key3": [100, 200, 300, 400, 500]})

df7 = pd.DataFrame({"key1": [1, 2, 3, 4, 5], "key2": [100, 200, 300, 400, 500]})

df8 = pd.DataFra

In [7]:
trainer.save_model("Llama-3.2-1B-Instruct-Batch-2-Step-60-lr-2e_4")

In [ ]:
# torch.save(model.state_dict(), 'llama_trained_model.pth')

In [ ]:
# model.save_pretrained("llama_trained_model")
# tokenizer.save_pretrained("llama_trained_model")

('llama_trained_model/tokenizer_config.json',
 'llama_trained_model/special_tokens_map.json',
 'llama_trained_model/tokenizer.json')